> **How to run this notebook (command-line)?**
1. Install the `ReinventCommunity` environment:
`conda env create -f environment.yml`
2. Activate the environment:
`conda activate ReinventCommunity`
3. Execute `jupyter`:
`jupyter notebook`
4. Copy the link to a browser


# `REINVENT 2.0`: reinforcement learning with tanimoto similarity


This is a simple example of running `Reinvent` with only 1 score component.

NOTE: There is a detailed reasoning for each code block provided in the `Reinforcement Learning Demo` notebook.


## 1. Set up the paths
_Please update the following code block such that it reflects your system's installation and execute it._

In [13]:
# load dependencies
import os
import re
import json
import tempfile

# --------- change these path variables as required
reinvent_dir = os.path.expanduser("~/Desktop/Projects/Publications/2020/2020-04_REINVENT_2.0/Reinvent")
reinvent_env = os.path.expanduser("~/miniconda3/envs/reinvent_shared.v2.1")
output_dir = os.path.expanduser("~/Desktop/REINVENT_RL_demo")

# --------- do not change
# get the notebook's root path
try: ipynb_path
except NameError: ipynb_path = os.getcwd()

# if required, generate a folder to store the results
try:
    os.mkdir(output_dir)
except FileExistsError:
    pass

## 2. Setting up the configuration 
In the cells below we will build a nested dictionary object that will be eventually converted to JSON file which in turn will be consumed by `REINVENT`. 
You can find this file in your `output_dir` location.

### A) Declare the run type

In [14]:
# initialize the dictionary
configuration = {
    "version": 2,                          # we are going to use REINVENT's newest release
    "run_type": "reinforcement_learning"   # other run types: "sampling", "validation",
                                           #                  "transfer_learning",
                                           #                  "scoring" and "create_model"
}

### B) Sort out the logging details
This includes `resultdir` path where the results will be produced.

Also: `REINVENT` can send custom log messages to a remote location. We have retained this capability in the code. if the `recipient` value differs from `"local"` `REINVENT` will attempt to POST the data to the specified `recipient`. 

In [15]:
# add block to specify whether to run locally or not and
# where to store the results and logging
configuration["logging"] = {
    "sender": "http://0.0.0.1",          # only relevant if "recipient" is set to "remote"
    "recipient": "local",                  # either to local logging or use a remote REST-interface
    "logging_frequency": 10,               # log every x-th steps
    "logging_path": os.path.join(output_dir, "progress.log"), # load this folder in tensorboard
    "resultdir": os.path.join(output_dir, "results"),         # will hold the compounds (SMILES) and summaries
    "job_name": "Reinforcement learning demo",                # set an arbitrary job name for identification
    "job_id": "demo"                       # only relevant if "recipient" is set to a specific REST endpoint
}

Create `parameters` field:

In [16]:
# add the "parameters" block
configuration["parameters"] = {}

### C) Set Diversity Filter
During each step of Reinforcement Learning the compounds scored above `minscore` threshold are kept in memory. Those scored smiles are written out to a file in the results folder `scaffold_memory.csv`.

In [17]:
# add a "diversity_filter"
configuration["parameters"]["diversity_filter"] =  {
    "name": "IdenticalMurckoScaffold",     # other options are: "IdenticalTopologicalScaffold", 
                                           #                    "NoFilter" and "ScaffoldSimilarity"
                                           # -> use "NoFilter" to disable this feature
    "nbmax": 25,                           # the bin size; penalization will start once this is exceeded
    "minscore": 0.4,                       # the minimum total score to be considered for binning
    "minsimilarity": 0.4                   # the minimum similarity to be placed into the same bin
}

### D) Set Inception
* `smiles` provide here a list of smiles to be incepted 
* `memory_size` the number of smiles allowed in the inception memory
* `sample_size` the number of smiles that can be sampled at each reinforcement learning step from inception memory

In [18]:
# prepare the inception (we do not use it in this example, so "smiles" is an empty list)
configuration["parameters"]["inception"] = {
    "smiles": [],                          # fill in a list of SMILES here that can be used (or leave empty)
    "memory_size": 100,                    # sets how many molecules are to be remembered
    "sample_size": 10                      # how many are to be sampled each epoch from the memory
}

### E) Set the general Reinforcement Learning parameters
* `n_steps` is the amount of Reinforcement Learning steps to perform. Best start with 1000 steps and see if thats enough.
* `agent` is the generative model that undergoes transformation during the Reinforcement Learning run.

We reccomend keeping the other parameters to their default values.

In [19]:
# set all "reinforcement learning"-specific run parameters
configuration["parameters"]["reinforcement_learning"] = {
    "prior": os.path.join(ipynb_path, "models/augmented.prior"), # path to the pre-trained model
    "agent": os.path.join(ipynb_path, "models/augmented.prior"), # path to the pre-trained model
    "n_steps": 125,                        # the number of epochs (steps) to be performed; often 1000
    "sigma": 128,                          # used to calculate the "augmented likelihood", see publication
    "learning_rate": 0.0001,               # sets how strongly the agent is influenced by each epoch
    "batch_size": 128,                     # specifies how many molecules are generated per epoch
    "reset": 0,                            # if not '0', the reset the agent if threshold reached to get
                                           # more diverse solutions
    "reset_score_cutoff": 0.5,             # if resetting is enabled, this is the threshold
    "margin_threshold": 50                 # specify the (positive) margin between agent and prior
}

### F) Define the scoring function
We will use only a `tanimoto_smiliarity` component with only one SMILES string:

`"O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N"`

However, using multiple smiles strings is also acceptable.

In [20]:
# prepare the scoring function definition and add at the end
scoring_function = {
    "name": "custom_product",                  # this is our default one (alternative: "custom_sum")
    "parallel": False,                         # sets whether components are to be executed
                                               # in parallel; note, that python uses "False" / "True"
                                               # but the JSON "false" / "true"

    # the "parameters" list holds the individual components
    "parameters": [

    # add component: use 
    {
        "component_type": "tanimoto_similarity", 
        "name": "Tanimoto similarity",         # arbitrary name for the component
        "weight": 1,                           # the weight of the component (default: 1)
        "model_path": None,                    # not required; note, this is "null" in JSON
        "smiles": ["O=S(=O)(c3ccc(n1nc(cc1c2ccc(cc2)C)C(F)(F)F)cc3)N"], # a list of SMILES can be provided
        "specific_parameters": None            # not required; note, this is "null" in JSON
    }]
}
configuration["parameters"]["scoring_function"] = scoring_function

## 3. Write out the configuration

We now have successfully filled the dictionary and will write it out as a `JSON` file in the output directory. Please have a look at the file before proceeding in order to see how the paths have been inserted where required and the `dict` -> `JSON` translations (e.g. `True` to `true`) have taken place.

In [21]:
# write the configuration file to the disc
configuration_JSON_path = os.path.join(output_dir, "RL_config.json")
with open(configuration_JSON_path, 'w') as f:
    json.dump(configuration, f, indent=4, sort_keys=True)

## 4. Run `REINVENT`
Now it is time to execute `REINVENT` locally. Note, that depending on the number of epochs (steps) and the execution time of the scoring function components, this might take a while. As we have only specified a low number of epochs (125) and all components should be fairly quick, this should not take too long in our case though.

The command-line execution looks like this:
```
# activate envionment
conda activate reinvent_shared.v2.1

# execute REINVENT
python <your_path>/input.py <config>.json
```

In [22]:
%%capture captured_err_stream --no-stderr

# execute REINVENT from the command-line
!python {reinvent_dir}/input.py {configuration_JSON_path}

In [23]:
# print the output to a file, just to have it for documentation
with open(os.path.join(output_dir, "run.err"), 'w') as file:
    file.write(captured_err_stream.stdout)

# prepare the output to be parsed
list_epochs = re.findall(r'INFO.*?local', captured_err_stream.stdout, re.DOTALL)
data = [epoch for idx, epoch in enumerate(list_epochs) if idx in [1, 75, 124]]
data = ["\n".join(element.splitlines()[:-1]) for element in data]

We have calculated a total of 125 epochs, let us quickly investigate how the agent fared. Below you see the print-out of the first, one from the middle and the last epoch, respectively. Note, that the fraction of valid `SMILES` is high right from the start (because we use a pre-trained prior). You can see the partial scores for each component for the first couple of compounds, but the most important information is the average score. You can clearly see how it increases over time.

In [24]:
for element in data:
    print(element)

INFO     
 Step 0   Fraction valid SMILES: 97.7   Score: 0.1951   Time elapsed: 0   Time left: 0.0
  Agent     Prior     Target     Score     SMILES
-36.89    -36.89    -36.89      0.00      c1ccc(CCc2c(-n3nncc3)cc(C)c(Cl)c2)c2[nH]c(C)c(CC)c(=O)n12
-35.77    -35.77     -8.39      0.21      S(=O)(=O)(c1ccc(C)cc1)N1C(C(NC(CC(=O)O)c2ccc(OC3CCNC3)cc2)=O)CC(OC)C1
-37.16    -37.16    -11.84      0.20      c1c2oc(-c3ccc(OC)cc3)cc2c(C2=CCC(N3CCN(c4ncccn4)CC3)CC2)cc1OC
-24.57    -24.57      4.91      0.23      c1cccc(-n2[nH]c(C(N3CCC(CN)CC3)=O)cc2=NC(=O)c2cc(-c3ncccc3)c(F)cc2Cl)c1
-25.29    -25.29     -1.42      0.19      C(F)(F)(F)Sc1c(NC(NCC)=S)cccc1
-33.76    -33.76      0.90      0.27      c1c(S(=O)(N(c2ccc(OC)cc2)C(C)C)=O)cc(C(=O)OCC(C(F)F)(F)F)cc1
-33.19    -33.19     -9.19      0.19      C(=O)(Nc1ccccc1OC(c1cccc(OC)c1)=O)NCCCCCCCC
-18.88    -18.88     16.57      0.28      C1(c2ccccc2)=NCc2n(c(Br)nn2)-c2c1cc(Cl)cc2
-36.45    -36.45     -4.03      0.25      FC(F)(F)c1c2nc(=Nc3ccc(C(NCC(C)(

## 5. Analyse the results
In order to analyze the run in a more intuitive way, we can use `tensorboard`:

```
# go to the root folder of the output
cd <your_path>/REINVENT_RL_demo

# make sure, you have activated the proper environment
conda activate reinvent_shared.v2.1

# start tensorboard
tensorboard --logdir progress.log
```

Then copy the link provided to a browser window, e.g. "http://workstation.url.com:6006/". The following figures are exmaple plots - remember, that there is always some randomness involved. In `tensorboard` you can monitor the individual scoring function components. What you see is, that all of those depicted went up (and `Fraction_valid_SMILES` was high troughout). Not shown is the predictive model, which did not perform all that well, so you might want to consider a higher weight next time.

![](img/individual_components.png)

Also the total score increased over time.

![](img/total_score.png)

It might also be informative to look at the results from the prior (dark blue), the agent (blue) and the augmented likelihood (purple) over time.

![](img/likelihood.png)

And last but not least, there is a "Images" tab available that lets you browse through the compounds generated in an easy way. In the molecules, the substructure matches that were defined to be required are highlighted in red (if present). Also, the total scores are given per molecule.

![](img/molecules.png)

The results folder will hold four different files: the agent (pickled), the input JSON (just for reference purposes), the memory (highest scoring compounds in `CSV` format) and the scaffold memory (in `CSV` format).

In [25]:
!head -n 15 {output_dir}/results/memory.csv

,smiles,score,likelihood
46,FC(F)(F)c1nn(-c2ccc(S(=O)(=O)N)cc2)c(-c2ccc(C)cc2)c1,1.0,-17.600012
61,S(=O)(=O)(c1ccc(-n2nc(C(F)(F)F)cc2-c2ccc(C)cc2)cc1)C,0.84615386,-18.967031
62,c1cc(C(=S)N)ccc1-n1nc(C(F)(F)F)cc1-c1ccc(C)cc1,0.82417583,-23.19146
125,C(c1cc(-c2ccccc2)n(-c2ccc(S(N)(=O)=O)cc2)n1)(F)(F)F,0.82417583,-17.847729
59,c1(-c2n(-c3ccc(S(N)(=O)=O)cc3)nc(C(F)(F)F)c2)ccc(Cl)cc1,0.8064516,-17.33696
19,c1cc(-n2c(-c3ccc(C)cc3)cc(C(F)(F)F)n2)ccc1[N+](=O)[O-],0.7291667,-23.365183
112,c1(-c2ccncc2)n(-c2ccc(S(N)(=O)=O)cc2)nc(C(F)(F)F)c1,0.7113402,-19.437431
1,c1(-c2ccc(-c3cc(C(F)(F)F)nn3-c3ccc(S(N)(=O)=O)cc3)cc2)ccccc1C,0.70535713,-23.20647
118,c1(-c2cc([N+]([O-])=O)ccc2C)n(-c2ccc(S(N)(=O)=O)cc2)nc(C(F)(F)F)c1,0.7019231,-25.672398
11,c1cc(-c2n(-c3ccc(S(=O)(=O)C)cc3)nc(C(F)(F)F)c2)ccc1F,0.6969697,-18.464922
69,O(c1c(OC)cc(-c2n(-c3ccc(S(=O)(N)=O)cc3)nc(C(F)(F)F)c2)cc1)C,0.6952381,-21.52629
2,c1(-c2ccc(-c3nnn(C)n3)cc2)cc(C(F)(F)F)nn1-c1ccc(S(=O)(=O)N)cc1,0.6909091,-26.232084
101,c1(-c2cc(C(F)(F